In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np

In [11]:
from experiments import OptimalSensorChoice
from experiments import utilities as utils
from bayesinverse import Regression
from robiplotipy import PlotEnv
from robiplotipy.experiments.optimal_sensor_choice import (
    plot_pcm_rmse,
    plot_pcm_information,
)


In [4]:
config_path = config_path = Path(
    "/home/rmaiwald/code/Experiments/experiments/optimal_sensor_choice/config.yaml"
)

exp = OptimalSensorChoice(config_path)


sensors 0.003113463521003723
emissions 1.5957173556089401
transport 4.198032721877098


In [15]:
exp.K = exp.transport.get_transport(
    exp.sensors.n_sensors, exp.sensors.get_index(), exp.emissions, n_processes=30,
)


loop start 12.547204047441483
loop end 12.587120234966278


In [16]:
true_emissions = exp.emissions.get_absolute()[exp.emissions.mask]


In [17]:
n_mc = 100
n_time = 24

n_sensors_list = [5, 10, 15, 20, 25, 30]
std_list = [0.1, 0.5, 1.0, 2.0, 3.0, 4.0, 5.0, 10.0]
rmse_list = []
information_list = []
for n_sensors_sample in n_sensors_list:
    print(n_sensors_sample)
    for std in std_list:
        for i in range(n_mc):
            time_ids = exp.transport.get_time_ids(time=n_time)
            sensor_ids = exp.sensors.get_sample_ids(n_sensors_sample)
            exp.sensors.set_std(std)
            reg = Regression(
                y=utils.stack_xr(
                    exp.K.isel(time_measurement=time_ids, sensor=sensor_ids)
                    @ exp.emissions.truth
                    + exp.sensors.get_noise(n_sensors_sample).isel(
                        time_measurement=time_ids
                    )
                ).values,
                K=utils.stack_xr(
                    exp.K.isel(time_measurement=time_ids, sensor=sensor_ids)
                ).values,
                x_prior=utils.stack_xr(exp.emissions.prior).values,
                x_covariance=utils.stack_xr(exp.emissions.prior_covariance).values,
                y_covariance=utils.stack_xr(
                    exp.sensors.get_covariance().isel(
                        time_measurement=time_ids, sensor=sensor_ids
                    )
                ).values,
            )
            information_list.append(reg.get_information_content())
            x_est, _, _, _ = reg.fit()
            x_posterior = exp.emissions.to_xr(x_est)
            x_posterior = utils.unstack_xr(x_posterior) * true_emissions
            # error = exp.emissions.truth_absolute * (exp.emissions.truth - x_posterior)
            # rmse = np.sqrt(np.mean(error**2))
            rmse = np.sqrt(
                np.mean(
                    (
                        exp.emissions.truth_absolute.sum(dim="source_group")
                        - x_posterior.sum(dim="source_group")
                    )
                    ** 2
                )
            )
            rmse_list.append(rmse)

std_list.reverse()
n_sensors_list.reverse()


5
10
15
20
25
30


In [18]:
rmse_array = np.array(rmse_list).reshape(len(n_sensors_list), len(std_list), n_mc)
rmse_array = np.flip(rmse_array.mean(axis=2))
rmse_array = rmse_array / np.sum(exp.emissions.truth_absolute).values * 100
rmse_array.min()

3.466880132073212

In [19]:
information = np.array(information_list).reshape(len(n_sensors_list), len(std_list), n_mc)
information = np.flip(information.mean(axis=2))


Saving

In [21]:
exp.data["std_list"] = std_list
exp.data["n_sensors_list"] = n_sensors_list
exp.data["rmse_list"] = rmse_list
exp.data["rmse_array"] = rmse_array
exp.data["information"] = information
exp.pickle_data()